# Team 3-3: Phase 1 EDA - Predicting Flight Delays to Mitigate Potential Delays and Costs

## Phase Leader Plan
| Week | Date | Phase | Owner | Deliverable Due |
|-----| ----- | ----- | ----- | ----- |
| Week 1 | Oct 27 | Phase 1 | Jason Dong | Nov 4
| Week 2 | Nov 3 | Phase 2 | Anson Quon |
| Week 3 | Nov 10 | Phase 2 | Nick Gasser |
| Week 4 | Nov 17 | Phase 2 | Gilbert Wong | Nov 24
| Week 5 | Nov 24 | Phase 3 | Sameer Karim |
| Week 6 | Dec 1 | Phase 3 | Jason Dong | Dec 8

## Project Abstract

Airline On-Time Performance, defined as a flight arriving within 15 minutes of expected arrival time, is a critical key performance indicator airlines and regulators track across the flight industry. The TranStats flight data from the US Department of Transportation, USDOT,  indicates 18% of flights were delayed by more than 15 minutes between 2015-2021, illustrating room for improvement across the industry. Each minute a flight is delayed can lead to escalating operational costs, especially with the USDOT’s new ruling requiring airline companies to provide automatic refunds for any domestic flights delayed three hours or more.

The goal of our classification model is to predict if a flight will be delayed two hours prior to the flight's expected departure time. Flight data from TranStats will be used to identify effects of flight delays across the network. Additionally, we will supplement flight information with weather data from the National Oceanic and Atmospheric Administration to further determine temporal trends and impacts to delays. Our machine learning pipeline includes checkpoints for feature extraction and feature engineering to prevent data leakage, create temporal and graph features, and appropriately balancing and splitting our feature set prior to model training. We will use logistic regression and random forests as our baseline due to their intepretability before experimenting with other algorithms. Precision, recall, and f1 score will be used to evaluate and select our optimal model. Once our optimal is identified, we look to further enhance our predictions with the estimated time of delay and type of delay to allow airline companies to prioritize and mitigate potential delays and costs.


## Data


### Description of Data

The data for the project will be sourced from three different datasets.
  - The first is flight information from the US Department of Transportation (DOT). This contains 109 features and ~31.7 million rows. It contains information related to the flights such as departure and arrival destinations, fligth durations (taxi and flight times), carrier information, distance traveled, and whether the flight was delayed or diverted. This data will be limited to US states and territories for depature and arrival locations. The full dataset contains information from 2015 to 2021.
  - The second dataset contains weather information from the National Oceanic and Atmospheric Administration. This contains 177 features and ~630.9 million rows. It contains various weather information such as temperature, humidity, precipitation, visiblity, sunrise time, and sunset time. The location information and time information can be used to join with the flight dataset to gain information on potential weather features impacting flight delays. The full dataset contains information from 2015 to 2021.
  - The final dataset contains airport information from the US DOT. This contains 10 features and ~18K rows. It contains location information for airports which can be used to merge the weather and flight datasets.

The data has been split into various time intervals to perform initial model development. This will allow for testing and working on scalability of the process. For the flight and weather data there are splits for Q1 2015 (3 months of data), Q1 and Q2 2015 (6 months of data), 2019 (1 year of data), and the full datasets from 2015-2021.

Based on the scope of the project, predicting a 15 minute delay two hours before a flight. Data will need to be filtered and masked while training the model to avoid data leak. Information that would not be available two hours before a flight, such as actual departure time of a flight or a delay reason will be masked. However, the data source contains complete flight information for the day and the flight numbers to track the planes movement. A previous flights information for a plane will be utilized, such as actual departure time or delay information will be utilized (if it is available prior to two hours of the current flight).

### Data Dictionary
The data from each source has been narrowed down to useful features. Please see the data dictionary to see the complete list of features for each dataset, which features could be used with model development, and the plan to merge the data together. [(Data Dictionary)](https://docs.google.com/spreadsheets/d/1cxMpgoy3YIUD1OGv9_BM3s-Q6DRTpuuF_pKyUKDrXJc/edit?gid=0#gid=0).

Key data elements to support our model prediction are listed below:
| Data Element | Objective |
| ----- | ----- |
| <b>Flight Data</b> | |
| Reporting Airline and Flight Number | Identifies a flight route with airline company between airports |
| Tail Number | Identifies an airline enabling the reconstruction of a plane's flight history |
| City, State, Latitude, Longitude | Filter data to US and US territories and connect with weather data |
| Destination and Arrival Airport | Basis to create graph features to measure an airport's influence on flight network |
| <b> Preventing Data Leakage </b> | |
| Departure and Arrival Time | Exclude data within two hours of expected departure time
| Weather Reading DateTime | Exclude weather data within two hours of expected departure time
| <b> Prediction Objectives </b> | |
| Departure Delay Indicator | Primary prediction: boolean indicator if a departure was delayed 15 minutes or more |
| Departure Delay Group | Delay time grouping in 15 minute intervals |
| Carrier Delay | Indicator if delay was due to carrier |
| Weather Delay | Indicator if delay was due to weather |



### Initial EDA

As mentioned earlier, the data was explored to gain an understanding of which features could potentially be used, potential feature correlation, and data distribution.
  - The project required use of US state and territory flights (departure and arrival). The flights dataset was reviewed and only contains US state and territory depatures and arrivals.
  - Upon reviewing the data it was determined the predicted outcome variable (delayed more than 15 minutes) is skewed, 82% of the flights were on time and 18% were delayed. This makes sense as most flights aren't delayed. Models and evaluation metrics will need to account for this skew in the outcome variable.
  - There was also a right skew in the data for the delay times. The average delay time was 9.2 minutes but the median was -2.0 minutes (early). The minimum delay time was -29 minutes and the maximum was 1,175 minutes.







<img src="https://raw.githubusercontent.com/ngasserberk/mids-w261-final_project/refs/heads/main/delay_dist.png?token=GHSAT0AAAAAACZ5NDIXRVHTSCX3KMVXXOM2ZZIBO2Q">

  - To gain an understanding if there was a uniform distribution of flights, the count of flights and percentage of delayed flights by year, month, and day of week were reviewed on a sample of the full dataset (2015-2021). 
    - In 2018 and 2019, the count of flights began to increase before drastically decreasing in 2020 and 2021, likely due to COVID-19 pandemic.
    - There was a smaller number of delayed flights in 2020 compared to other years (2021 was similar to 2015-2019).
    - There appears to be seasonality across months of the data. Similar behavior occurs throughout the week, with a lower percent of delays on Monday and Tuesday.

<img src="https://raw.githubusercontent.com/ngasserberk/mids-w261-final_project/refs/heads/main/seasonality_flight_count.png?token=GHSAT0AAAAAACZ5NDIWIWUK3SDLMXMPZBESZZIBAKQ">

<img src="https://raw.githubusercontent.com/ngasserberk/mids-w261-final_project/refs/heads/main/seasonality_delay_perc.png?token=GHSAT0AAAAAACZ5NDIXY3M4UUICCHGHK32OZZH756Q">

  - The distribution of flights by airlines was reviewed and the distribution of delay time.
    - The most flights are with WN. The majority of the flights are by WN, DL, AA, OO, and UA. Following those, there are 15 other airlines with fewer flight counts.
    - There does not seem to be a destinct correlation between number of flights for an airline and delay time. As shown earlier, there is a large skew in the delay times for each airline. The figure was restricted to a maximum delay time of 100 minutes while we saw earlier the max delay was 1,175 minutes.
    - Airlines F9, B6, QX, and WN appear to have the widest distribution of delay times.

<img src="https://raw.githubusercontent.com/ngasserberk/mids-w261-final_project/refs/heads/main/flights_by_airline.png?token=GHSAT0AAAAAACZ5NDIXKNYC4XPZO64QJ2XIZZH75QQ">

<img src="https://raw.githubusercontent.com/ngasserberk/mids-w261-final_project/refs/heads/main/airline_box_delays.png?token=GHSAT0AAAAAACZ5NDIWOKQRLZ45OTYHZM6OZZIBQLQ">

  - A Pearson correlation between numeric features were reviewed. Highly correlated features were reviewed. These should be noted for the models to avoid feature collinearity.
    - Flight features such as flight time with distance and departure delay with arrival delay have a strong positive correlation.
    - Weather features such as sea level pressure with altimeter setting, sky conditions with preciptiation, wet bulb temperature with dew point temperature and dry bulb temperature have a strong positive correlation. Features station pressure with elevation, sky conditions with altimeter setting, sky conditions with relative humidity has a strong negative correlation.




%md
<img src="https://raw.githubusercontent.com/ngasserberk/mids-w261-final_project/refs/heads/main/num_corr.png">

### Missing Data 

Missing data at the feature level was reviewed. The count and percent of non-null values were reviewed for each dataset
  1. Initial quick analysis of the flights dataset was reduced to remove any feature that had less than 15% percent of non-null values. While, normally you wouldn't want to include features that sparse, some are only filled for delayed flights, such as the delay indicators (weather, carrier, etc.). Thus, these are only ~18% filled. The remaining features with less than 15% filled values were dropped, 48. This included flight information such as grounded time away from gate, and flight deviation information.
  2. The weather dataset was reviewed and determined to be sparse for the majority of the features. However, this dataset is at the latitude and longitude level. Thus, taking a direct reduction from the full table wouldn't make sense as some coordinates won't join with the flight data. Missing values will be evaluated further after joining the data.

Our plan to address missing data is categorized into the following buckets:
- <b> Delay Data </b> - Data specific to delayed flights will be filled with a generic value for on time flights.
- <b> Weather / Temporal Data </b> - Linear or quadratic interpolation following further assessment of trends and time gaps.
- <b> Numerical Data </b> - Impute mean, median, or mode depending on the distribution of data.
- <b> Categorical Flight Data </b> - Carry last observation forward to maintain continuity of data per tail aircraft.

## Machine Learning Algorithms and Metrics
For this project, we are focusing on predicting **departure delays** where a delay is defined as being 15 minutes or greater past the planned departure time. The prediction will be made at least **two hours before departure** to allow sufficient time for airlines and airports to notify passengers and adjust operations. Our primary stakeholders include airlines, airports, and passengers. This is framed as a **classification problem**, where the target variable is whether a flight will be delayed or not.

Our baseline and model iterations to predict departure delays will focus on the following algorithms:

1. **Logistic Regression**:
   - **Implementation**: Using `PySpark`'s `LogisticRegression` class.
   - **Loss Function**: Binary Cross-Entropy Loss:
     $$
     L = -\frac{1}{N} \sum_{i=1}^{N} [y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i)]
     $$
   - **Reasoning**: A simple baseline to understand feature importance and build interpretability.

2. **Random Forest Classifier**:
   - **Implementation**: Using `PySpark`'s `RandomForestClassifier`.
   - **Feature Importance**: Helps identify critical factors contributing to delays.
   - **Advantage**: Good for capturing non-linear relationships and robust to overfitting with proper tuning while providing high interpretability.

3. **Gradient Boosted Trees**:
   - **Implementation**: Using `PySpark`'s `SparkXGBRegressor`
   - **Loss Function**: Logistic loss for binary classification.
   - **Advantage**: Effective for handling imbalanced data and complex relationships.
   - **Reasoning**: Provides high predictive power while maintaining efficiency.

The method that can be used for cross-validating the time-series model is rolling cross-validation. This approach ensures that the temporal structure of the data is preserved, which is essential for time-series problems where past information should not have access to future data. This rolling cross-validation approach is particularly useful for ensuring that the model's predictions remain robust over time and that it adapts to changes or trends in the data. Additionally, it helps prevent data leakage by maintaining the correct sequence of training and validation periods.

To evaluate the performance of our models, we will use a combination of the following metrics: 

1. **Precision**:
   $$
   \text{Precision} = \frac{\text{True Positives}}{\text{True Positives} + \text{False Positives}}
   $$
   Indicates how many of our predicted delays are actual delays to avoid false alerts to airline companies. Higher precision may lead to missed predictions.

2. **Recall**:
   $$
   \text{Recall} = \frac{\text{True Positives}}{\text{True Positives} + \text{False Negatives}}
   $$
   Essential for identifying the most delays to minimize unexpected delays. Higher recall may lead to incorrectly identifying flight delays.

3. **F1-Score**:
   $$
   \text{F1-Score} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
   $$
   Provides a balance between precision and recall, crucial for understanding the model's performance in the context of both metrics.

![](path)

## Machine Learning Pipeline

Our current machine learning pipeline and checkpointing strategy is as follows:

![pipeline](https://github.com/jasondongmids/mids_w261_final_project/blob/main/ref/ML%20pipeline2.png?raw=true)

We begin our EDA and feature engineering on a subset of 3-month and 1-year flight data prior to expanding to the entire dataset. To ensure modularity, rapid prototyping, and failsafes against disruptions, we plan these checkpoints up to model training:
- <b> Data Ingestion</b> - Sync various data formats to Parquest for better utilization of distributed resources.
- <b> Data Dictionary and Feature Extraction </b> - Identify scope of data elements for model prediction and data type transformations.
- <b> Feature Engineering </b> - Complete tempoeral transformations, graph transformations, normalization, and derivations as a starting basis for model development and training.
- <b> Model Training </b> - Model architecture, callbacks, early stopping, and model saving will be employed to guard against training disruptions and provide a quick restart to downstream predictions and metric analyses.



## Team Members

- Jason Dong 
- Nick Gasser 
- Sameer Karim 
- Anson Quon 
- Gilbert Wong


## Conclusions and Next Steps
To enable airline companies to mitigate costs from flight delays, we will create a classification model to predict if a flight will be delayed two hours prior to it's expected departure. Our initial EDA determined key data elements, methods to prevent data leakage, feature engineering requirements, and options for data splitting for our data pipeline. In the next phase, we will use the groundwork EDA in this phase to complete our data pipeline and assess our model baselines via random forest and logistic regression algorithms before further refinement to select the optimal model and further experimentation for additional time of delay and type of delay predictions.

### Open Issues and Problems
- EDA on COVID's potential impact on the recent end of our data and the impact to feature engineering and data splitting.
- Scope requirements for custom data model.